In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [5]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
# bets.drop([6], inplace = True)
# bets.reset_index(inplace = True, drop = True)
# bets.to_csv('In_Season/Data/todays_bets.csv')

In [7]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
# results = results[results.Date != '2022-02-15']
# results.to_csv('In_Season/Data/results_tracker.csv')

In [19]:
external_bets = pd.read_csv('In_Season/Data/todays_bets_external.csv')

In [2]:
tables = pd.read_html('https://www.espn.com/nba/injuries')

# Iterating through injury tables to create injury list
injuries = pd.DataFrame(columns = ['NAME', 'STATUS'])
for table in tables:
    table = table[['NAME', 'STATUS']]
    injuries = injuries.append(table)
injuries.reset_index(drop = True, inplace = True)

# Filtering out players who may not be out
injuries = injuries[injuries.STATUS == 'Out']

# Adjusting columns
injuries = injuries[['NAME']]
injuries.columns = ['Player']

In [3]:
def calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

# Creating team map to align w/ final output
team_map = {
    'Wizards' : 'Washington Wizards',
    'Celtics' : 'Boston Celtics',
    'Pelicans' : 'New Orleans Pelicans',
    'Knicks' : 'New York Knicks',
    'Pistons' : 'Detroit Pistons',
    'Magic' : 'Orlando Magic',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Pacers' : 'Indiana Pacers',
    'Raptors' : 'Toronto Raptors',
    'Grizzlies' : 'Memphis Grizzlies',
    'Heat' : 'Miami Heat',
    'Bulls' : 'Chicago Bulls',
    'Jazz' : 'Utah Jazz',
    'Bucks' : 'Milwaukee Bucks',
    'Spurs' : 'San Antonio Spurs',
    'Warriors' : 'Golden State Warriors',
    'Thunder' : 'Oklahoma City Thunder',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Nuggets' : 'Denver Nuggets',
    'Suns' : 'Phoenix Suns',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Kings' : 'Sacramento Kings',
    'Hornets' : 'Charlotte Hornets',
    'Trail Blazers' : 'Portland Trailblazers',
    'Nets' : 'Brooklyn Nets',
    'Lakers' : 'Los Angeles Lakers',
    'Clippers' : 'Los Angeles Clippers',
    'Rockets' : 'Houston Rockets'
}

# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nba/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting table from HTML
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds.dropna(inplace = True)
odds.reset_index(drop = True, inplace = True)

odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():

    # Retreiving teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]

    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 12:
        ml_string = ml_string.replace('ML', '')
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 13:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    if len(ml_string) == 14:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[5] == '+') | (ml_string[5]=='-'):
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [4]:
odds_df

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Cleveland Cavaliers,Minnesota Timberwolves,140.0,-166.0,41.666667,62.406015
1,Orlando Magic,Indiana Pacers,-116.0,-102.0,53.703704,50.495050
2,Miami Heat,Chicago Bulls,-188.0,156.0,65.277778,39.062500
3,Oklahoma City Thunder,Sacramento Kings,180.0,-215.0,35.714286,68.253968
4,Milwaukee Bucks,Charlotte Hornets,-550.0,410.0,84.615385,19.607843
5,Memphis Grizzlies,San Antonio Spurs,-390.0,310.0,79.591837,24.390244
